# INST326 — Week 10 Exercises: Data Models, Properties, and Special Methods (Library Management)

**Focus (Week 10 only):** Python data model & class design refinements — `@property` (get/set/delete) with validation, dataclass options (equality, ordering, `field` options, `__post_init__`), special methods (`__str__`, `__repr__`, `__len__`, `__iter__`, `__contains__`, `__eq__`, ordering), and lightweight container patterns. Optional: `functools.total_ordering` and `dataclasses.replace`.

**Out of scope (Week 11+):** multiple inheritance/mixins, advanced design patterns, descriptors beyond `@property`, metaclasses, concurrency, ORMs, complex type-system features, network I/O, or frameworks.


### Starter Scaffold (Week-10-safe)

Below is minimal starter code from prior weeks, lightly adapted for Week 10. You may extend it while staying within the Week 10 scope.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field, replace
from datetime import datetime, timedelta
from typing import Dict, Iterable, Iterator, Optional, List
from functools import total_ordering

# --- Exceptions (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)

    # Comparable by title then ISBN (Week 10: ordering/special methods)
    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self.title, self.isbn) < (other.title, other.isbn)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn  # equality by identity key

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self.title!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self.title} [{self.isbn}]"

    # Week 10: properties for validation
    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @copies.deleter
    def copies(self) -> None:
        # Reset to zero (logical 'remove from circulation')
        self._copies = 0

@dataclass
class Member:
    member_id: str
    email: str

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog(Iterable[Book]):
    """Lightweight container with special methods (Week 10)."""
    def __init__(self):
        self._books: Dict[str, Book] = {}

    # Container-like dunder methods
    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        # Iterate in title order for determinism
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        # Allow 'in' checks by ISBN or Book
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    # API
    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=14)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Property validation

Add an `@property` named `title` to `Book` that strips whitespace on set and raises `ValueError` if empty after stripping. Keep the existing attribute name (override dataclass behavior carefully).

In [ ]:
# Your code here
# Hint: define a private _title and redirect in __post_init__ if needed.

## 2) Read-only computed property

Add a read-only property `label` on `Book` that returns `f"{title} [{isbn}]"`. Attempting to set `label` should raise `AttributeError`. Demonstrate usage.

In [ ]:
# Your code here

## 3) `__post_init__` data normalization

Use `__post_init__` on `Book` to normalize ISBN by stripping hyphens/spaces and uppercasing the title. Ensure validation runs on initial values too.

In [ ]:
# Your code here

## 4) `dataclasses.field` options

Modify `Book` so `_copies` is excluded from comparisons (`compare=False`) and included in `__repr__` via a custom `__repr__`. Explain briefly in a comment why we avoid comparing by copies.

In [ ]:
# Your code here

## 5) Ordering with `total_ordering`

Given `__eq__` (by ISBN) and `__lt__` (by title, then ISBN), verify that sorting a mixed list of `Book` objects orders by title. Show a quick sort demo.

In [ ]:
# Your code here

## 6) Container protocol methods

Implement `__delitem__(self, isbn: str)` on `Catalog` to remove a book or raise `KeyError` if missing. Show that `len(catalog)` updates accordingly.

In [ ]:
# Your code here

## 7) Slicing/lookup convenience

Implement `find_by_title_prefix(self, prefix: str) -> list[Book]` on `Catalog` that returns books whose title starts with the prefix (case-insensitive).

In [ ]:
# Your code here

## 8) Immutability via `replace`

Using `dataclasses.replace`, show how to create a new `Book` with a different title while leaving the original unchanged. Explain in a comment when immutability is helpful.

In [ ]:
# Your code here

## 9) Rich string representations

Customize `__repr__` and `__str__` for `Book` (or confirm from scaffold) and explain in a short comment how they help during debugging vs. user display.

In [ ]:
# Your code here

## 10) Truthiness protocol

Define `__bool__(self)` on `Book` so that a book evaluates to `True` if `copies > 0` and `False` otherwise. Show a one-line `if book:` demo.

In [ ]:
# Your code here

## 11) Hashability decision

Decide whether `Book` should be hashable based on ISBN. If yes, implement `__hash__` consistent with `__eq__`. If not, explain why and show how using `Book` as a `dict` key could be risky if mutable fields affect identity.

In [ ]:
# Your code here

## 12) In-place operations vs. new objects

Write a method `with_more_copies(self, n: int) -> Book` that returns a **new** `Book` with copies increased by `n` (do not mutate `self`). Show a quick before/after demonstration.

In [ ]:
# Your code here

## 13) Catalog iteration contract

Demonstrate that the provided `Catalog.__iter__` yields books in **sorted** order by title. Add an assertion-based test in a code cell.

In [ ]:
# Your code here

## 14) Membership semantics

Show that `isbn in catalog` and `book in catalog` both work via `__contains__`. Add a short test cell to verify both cases.

In [ ]:
# Your code here

## 15) Defensive copying in accessors

Add a method `to_list(self) -> list[Book]` to `Catalog` that returns a shallow copy of the book list (sorted). Explain why returning internal structures directly is risky.

In [ ]:
# Your code here

## 16) Lightweight value object

Create a small `@dataclass(frozen=True)` called `Author` with fields `last`, `first`. Add an optional `author: Author | None` to `Book` (default `None`). Show how `frozen=True` prevents later mutation and why that can be desirable for identity keys.

In [ ]:
# Your code here

## 17) Custom containment by predicate

Implement `contains_title(self, needle: str) -> bool` on `Catalog` that returns `True` if any book title equals `needle` case-insensitively. Do not modify `__contains__`.

In [ ]:
# Your code here

## 18) Pretty-print table helper

Write a standalone function `print_catalog_table(catalog: Catalog)` that prints a simple aligned table of ISBN, Title, Copies. Use only string formatting, no third-party libs.

In [ ]:
# Your code here

## 19) Sorting with key functions

Demonstrate sorting books by different keys without changing their natural order: by ISBN, by copies (descending), by title length. Use `sorted(..., key=...)`.

In [ ]:
# Your code here

## 20) Minimal unit tests for data model

Using `unittest`, write a small test case verifying:
- `Book.title` setter strips and forbids empty
- `Book.copies` setter validates
- `Catalog` iteration is sorted
- `__contains__` works for both ISBN and Book
(Keep it basic—no fixtures beyond a simple `setUp`.)

In [ ]:
# Your code here
import unittest

class TestWeek10DataModel(unittest.TestCase):
    def setUp(self):
        self.c = Catalog()
        self.c.add_book(Book("2","Beta",1))
        self.c.add_book(Book("1","Alpha",2))
    def test_title_property(self):
        b = Book("3","  New  ",1)
        # set title and ensure strip/validation
        ...
    def test_copies_validation(self):
        b = Book("4","Copies",1)
        ...
    def test_sorted_iteration(self):
        titles = [b.title for b in self.c]
        ...
    def test_contains(self):
        a = self.c.get_book("1")
        ...

# # To run in notebook:
# # unittest.main(argv=['-v'], exit=False)

## Python skills you'll need (Weeks 1–10)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts; simple comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write
- **Testing (Week 7):** `unittest.TestCase`, assertions, simple setup
- **OOP (Weeks 4–9):** classes, methods, inheritance (single), ABCs (light), polymorphism
- **Week 10 focus (data model):**
  - `@property` getters/setters/deleters
  - `dataclass` options: `field`, `__post_init__`, equality & ordering
  - Special methods: `__str__`, `__repr__`, `__len__`, `__iter__`, `__contains__`, `__eq__`, ordering (`total_ordering`)
  - Lightweight containers and defensive copying
  - Optional: `dataclasses.replace`
- **Standard library familiarity:** `dataclasses`, `functools.total_ordering`, `datetime`
